In [1]:
## Mount drive in the left tab to store the dataset and model info
%cd /content/drive/'My Drive'/

/content/drive/My Drive


In [0]:
# from google.colab import files
# files.upload() # to upload api token from kaggle to get dataset

In [0]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download bittlingmayer/amazonreviews
# ! unzip amazonreviews.zip -d amazon_reviews

In [0]:
import bz2
from collections import Counter
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import nltk
import torchtext
import numpy as np
import json
from torchtext import data
import spacy

In [0]:
train_file = bz2.BZ2File('./amazon_reviews/train.ft.txt.bz2')
test_file = bz2.BZ2File('./amazon_reviews/test.ft.txt.bz2')

In [0]:
train_file = train_file.readlines()
test_file = test_file.readlines()

In [5]:
print("Number of training reivews: " + str(len(train_file)))
print("Number of test reviews: " + str(len(test_file)))

Number of training reivews: 3600000
Number of test reviews: 400000


In [0]:
num_train = 100000 #We're training on the first 800,000 million reviews in the dataset
num_test = 20000 #Using 200,000 reviews from test set

train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]

In [0]:
train_labels = [0 if x.split()[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

# Further Splitting it down between validation and training set
train_sentences, val_sentences = train_sentences[int(0.2 * len(train_sentences)) : ], train_sentences[: int(0.2 * len(train_sentences))]
train_labels, val_labels = train_labels[int(0.2 * len(train_labels)) :], train_labels[: int(0.2 * len(train_labels))]

In [8]:
print ("Size of training data, ", len(train_sentences))
print ("Size of validation data, ", len(val_sentences))
print ("Size of testing data, ", len(test_sentences))

Size of training data,  80000
Size of validation data,  20000
Size of testing data,  20000


In [0]:
del train_file, test_file

In [0]:
# define the columns that we want to process and how to process
TEXT = data.Field(sequential=True, 
                       tokenize='spacy', 
                       include_lengths=True, 
                       use_vocab=True)
LABEL = data.Field(sequential=False, 
                         use_vocab=False, 
                         pad_token=None, 
                         unk_token=None)

In [0]:
fields = fields = [
    ('text', TEXT), 
    ('label', LABEL)
]

train_examples = [data.Example.fromlist([train_sentences[i], train_labels[i]], fields) 
                  for i in range(len(train_sentences))]
val_examples = [data.Example.fromlist([val_sentences[i], val_labels[i]], fields) 
                  for i in range(len(val_sentences))]
test_examples = [data.Example.fromlist([test_sentences[i], test_labels[i]], fields) 
                for i in range(len(test_sentences))]

In [0]:
train_data = data.Dataset(train_examples, fields)
val_data = data.Dataset(val_examples, fields)
test_data = data.Dataset(test_examples, fields)

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x : len(x.text),
    device = device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [23]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [24]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.4927,  0.2885,  1.1998,  ..., -0.2645, -0.7052, -1.1793],
        [-0.5954, -0.3418, -0.9033,  ..., -0.8614,  0.5441, -1.3190],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.8607, -0.0967, -0.0544,  ..., -0.0589, -0.2340,  1.0247],
        [ 0.0795,  0.5692,  0.0749,  ...,  0.4177,  0.6273,  0.0428],
        [-1.0707, -0.5379, -0.6604,  ...,  0.5555, -0.2943,  0.5282]])

In [25]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.8607, -0.0967, -0.0544,  ..., -0.0589, -0.2340,  1.0247],
        [ 0.0795,  0.5692,  0.0749,  ...,  0.4177,  0.6273,  0.0428],
        [-1.0707, -0.5379, -0.6604,  ...,  0.5555, -0.2943,  0.5282]])


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label.type_as(predictions))
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label.type_as(predictions))
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 2s
	Train Loss: 0.361 | Train Acc: 83.91%
	 Val. Loss: 0.261 |  Val. Acc: 89.10%
Epoch: 02 | Epoch Time: 1m 2s
	Train Loss: 0.227 | Train Acc: 91.00%
	 Val. Loss: 0.193 |  Val. Acc: 92.51%
Epoch: 03 | Epoch Time: 1m 1s
	Train Loss: 0.184 | Train Acc: 92.97%
	 Val. Loss: 0.185 |  Val. Acc: 92.71%
Epoch: 04 | Epoch Time: 1m 2s
	Train Loss: 0.160 | Train Acc: 93.94%
	 Val. Loss: 0.182 |  Val. Acc: 93.12%
Epoch: 05 | Epoch Time: 1m 2s
	Train Loss: 0.141 | Train Acc: 94.72%
	 Val. Loss: 0.193 |  Val. Acc: 93.09%


In [33]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.190 | Test Acc: 92.93%
